In [172]:
"""
write this in the Terminal:
conda install -c anaconda pandas-datareader

When asked:
The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates                                 pkgs/main --> anaconda
  certifi                                         pkgs/main --> anaconda
  openssl                                         pkgs/main --> anaconda
  qt                                              pkgs/main --> anaconda
Proceed ([y]/n)?

Press y

Install dash

jupyter nbextension enable --py widgetsnbextension

jupyter labextension install @jupyter-widgets/jupyterlab-manager

"""

import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import pandas as pd
import pandas_datareader as web
import numpy as np
import bs4 as bs
import pickle
import requests
import os
import pickle
import numpy as np
#import dash
#import dash_core_components as dcc
#import dash_html_components as html
from scipy.stats import norm # normal distribution
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
style.use("ggplot")

"""The purpose of the code below is to first get the tickers for each of the companites on the S&P500 index list, which we later on will
use to append all the historic data to the appropriate ticker. We use *requests.get* to access the data from Wikipedia, which is the list of tickers, after which we use *BeautifulSoup* to then read the HTML format of the website, and make a table. The *soup.find* command is used to specify where exactly in the HTML the table we want is located. 

We make an empty list, and use a for loop to read all entries in the column of the tickers and append each of these to the list we made (called tickers). 

We save it as a pickle as we want to access this data later on, and fast. We called this *"sp500tickers.pickle"*. """



'The purpose of the code below is to first get the tickers for each of the companites on the S&P500 index list, which we later on will\nuse to append all the historic data to the appropriate ticker. We use *requests.get* to access the data from Wikipedia, which is the list of tickers, after which we use *BeautifulSoup* to then read the HTML format of the website, and make a table. The *soup.find* command is used to specify where exactly in the HTML the table we want is located. \n\nWe make an empty list, and use a for loop to read all entries in the column of the tickers and append each of these to the list we made (called tickers). \n\nWe save it as a pickle as we want to access this data later on, and fast. We called this *"sp500tickers.pickle"*. '

The purpose of the code below is to first get the tickers for each of the companites on the S&P500 index list, which we later on will
use to append all the historic data to the appropriate ticker. We use *requests.get* to access the data from Wikipedia, which is the list of tickers, after which we use *BeautifulSoup* to then read the HTML format of the website, and make a table. The *soup.find* command is used to specify where exactly in the HTML the table we want is located. 

We make an empty list, and use a for loop to read all entries in the column of the tickers and append each of these to the list we made (called tickers). 

We save it as a pickle as we want to access this data later on, and fast. We called this *"sp500tickers.pickle"*. 



In [173]:
#Automating S&P500 - From Yahoo Finance - Close price adjusted for splits, and Adj. Close price is adjusted for both dividends and splits.
def save_sp500_tickers():
    resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find("table", {"class": "wikitable sortable"})
    tickers = []
    for row in table.findAll("tr")[1:]:
        ticker = row.findAll("td")[0].text.replace(".","-")
        ticker = ticker[:-1]
        tickers.append(ticker)

    df_tickers = pd.DataFrame(tickers)
    df_tickers.to_csv("tickers.csv")

    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    
        print(tickers)

        return(tickers)
    
save_sp500_tickers()

df_tickers = pd.read_csv("tickers.csv", index_col= 0)
df_tickers.columns = ["Ticker"]
df_tickers.head()

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'APC', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'ATO', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BHGE', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK-B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF-B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CBS', 'CE', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'COST', 'COTY', 'CCI', 'CS

,Ticker
0,MMM
1,ABT
2,ABBV
3,ABMD
4,ACN


Next, we need data for each ticker we have in the list. We get this from yahoo, and using the *DataReader* from pandas. 

Remember we saved the tickers as a pickle file, which we will now open and use to append the data to. Also, we make a separate folder for all of the CSV files we will get (505!). We define appropriate start and end dates for the data we want. 

We then create a for loop, and save each CSV file with price data in the stock_dfs folder. 

In [174]:

def get_names():
    resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find("table", {"class": "wikitable sortable"})
    names = []
    for row in table.findAll("tr")[1:]:
        name = row.findAll("td")[1].text.replace(".","-")
        names.append(name)

    df_names = pd.DataFrame(names)
    df_names.to_csv("names.csv")

    print(names)

get_names()

df_names = pd.read_csv("names.csv", index_col= 0)
df_names.columns = ["Name"]
df_names.head()


def get_sectors():
    resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find("table", {"class": "wikitable sortable"})
    sectors = []
    for row in table.findAll("tr")[1:]:
        sector = row.findAll("td")[3].text.replace(".","-")
        sectors.append(sector)

    df_sectors = pd.DataFrame(sectors)
    df_sectors.to_csv("sectors.csv")

    print(sectors)

get_sectors()

df_sectors = pd.read_csv("sectors.csv", index_col= 0)
df_sectors.columns = ["Sector"]
df_sectors.head()

df_names_and_sector = df_names.join(df_sectors, how = "right")
df_names_and_sector.head()

df_tickers_names_sectors = df_tickers.join(df_names_and_sector, how = "right")
df_tickers_names_sectors.head()
#df_tickers_names_sectors.set_index("Ticker", inplace=True)
#df_tickers_names_sectors.head()
df_tickers_names_sectors.values.tolist

with open ("tickers_names_sectors.pickle", "wb") as f:
    pickle.dump(df_tickers_names_sectors, f)

['3M Company', 'Abbott Laboratories', 'AbbVie Inc-', 'ABIOMED Inc', 'Accenture plc', 'Activision Blizzard', 'Adobe Systems Inc', 'Advanced Micro Devices Inc', 'Advance Auto Parts', 'AES Corp', 'Affiliated Managers Group Inc', 'AFLAC Inc', 'Agilent Technologies Inc', 'Air Products & Chemicals Inc', 'Akamai Technologies Inc', 'Alaska Air Group Inc', 'Albemarle Corp', 'Alexandria Real Estate Equities', 'Alexion Pharmaceuticals', 'Align Technology', 'Allegion', 'Allergan, Plc', 'Alliance Data Systems', 'Alliant Energy Corp', 'Allstate Corp', 'Alphabet Inc Class A', 'Alphabet Inc Class C', 'Altria Group Inc', 'Amazon-com Inc-', 'Ameren Corp', 'American Airlines Group', 'American Electric Power', 'American Express Co', 'American International Group', 'American Tower Corp-', 'American Water Works Company Inc', 'Ameriprise Financial', 'AmerisourceBergen Corp', 'AMETEK Inc-', 'Amgen Inc-', 'Amphenol Corp', 'Anadarko Petroleum Corp', 'Analog Devices, Inc-', 'ANSYS', 'Anthem Inc-', 'Aon plc', 'A-

In [175]:
#Getting data from Yahoo
def data_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2016, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

data_yahoo()


Already have MMM
Already have ABT
Already have ABBV
Already have ABMD
Already have ACN
Already have ATVI
Already have ADBE
Already have AMD
Already have AAP
Already have AES
Already have AMG
Already have AFL
Already have A
Already have APD
Already have AKAM
Already have ALK
Already have ALB
Already have ARE
Already have ALXN
Already have ALGN
Already have ALLE
Already have AGN
Already have ADS
Already have LNT
Already have ALL
Already have GOOGL
Already have GOOG
Already have MO
Already have AMZN
Already have AEE
Already have AAL
Already have AEP
Already have AXP
Already have AIG
Already have AMT
Already have AWK
Already have AMP
Already have ABC
Already have AME
Already have AMGN
Already have APH
Already have APC
Already have ADI
Already have ANSS
Already have ANTM
Already have AON
Already have AOS
Already have APA
Already have AIV
Already have AAPL
Already have AMAT
Already have APTV
Already have ADM
Already have ARNC
Already have ANET
Already have AJG
Already have AIZ
Already have A

Furthermore, the data from yahoo and the tickers are not very useful by themselves, so obvisouly we want to compile the data in order to get a dataframe with all tickers and their data. We open the pickle file again, and make an empty dataframe.

We can then compile each ticker with their data. We drop all columns that are not *Adj Close*, and rename *Adj Close* to the ticker name since we do not have any other data than for Adjusted Close price. We choose to only include this, since the Adjusted Close price takes into account payment of dividends of companies, eventual stock splits, and Rights offerings.

For easier comparison, we also index the data by dividing the first observation in each column by the rest of the column. 

We then use the empty data frame made before, and join all the data together into a single data frame. 

We also convert the data frame to a csv file for easy access to it. 

finally, we create a data frame from the CSV file just saved, and index the date. We call this new data frame *df_stocks*.

In [198]:
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    #Iterating though all DFs

    for count, ticker in enumerate(tickers):
        df = pd.read_csv("stock_dfs/{}.csv".format(ticker))
        df.set_index("Date", inplace=True)
        df.rename(columns = {"Adj Close": ticker}, inplace=True) #Adj Close takes the categories place in the column - Simple rename
        df.drop(["Open","High","Low","Close","Volume"],1, inplace=True)
    
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how="outer")
        
        if count % 100 == 0: #Only print #100, #200, #300, etc. - It will let you know how 
            print("I have now compiled", count, "stock files")
    print(main_df.head())
    main_df.to_csv("sp500_joined_adj_closes.csv")

compile_data()

data_df = pd.read_csv("sp500_joined_adj_closes.csv")
data_df = data_df.drop(["BHGE", "DWDP", "TPR", "ARNC", "ZBH", "OKE", "EVRG", "COST", "EW", "BBT", "JNJ", "VMC", "LIN", "COTY", "DGX", "ZBH", "FTV", "LW"], axis=1)
data_df.head()

I have now compiled 0 stock files
I have now compiled 100 stock files
I have now compiled 200 stock files
I have now compiled 300 stock files
I have now compiled 400 stock files
I have now compiled 500 stock files
                   MMM        ABT       ABBV       ABMD        ACN       ATVI  \
Date                                                                            
2016-01-04  135.163361  39.841354  50.214832  85.239998  95.316299  36.604977   
2016-01-05  135.752548  39.832066  50.005642  85.000000  95.812393  36.137928   
2016-01-06  133.018326  39.497967  50.014351  85.300003  95.625191  35.797367   
2016-01-07  129.777802  38.551361  49.866177  81.919998  92.817101  35.291401   
2016-01-08  129.335922  37.743946  48.506435  84.580002  91.918503  34.746506   

                 ADBE   AMD         AAP       AES    ...            WLTW  \
Date                                                 ...                   
2016-01-04  91.970001  2.77  151.373932  8.105658    ...      118.

,Date,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,WMB,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZION,ZTS
0,2016-01-04,135.163361,39.841354,50.214832,85.239998,95.316299,36.604977,91.970001,2.77,151.373932,...,21.791628,118.837708,64.482521,32.163685,25.023848,42.507366,34.595341,48.128719,25.348028,46.097820
1,2016-01-05,135.752548,39.832066,50.005642,85.000000,95.812393,36.137928,92.339996,2.75,150.339844,...,21.866369,119.655312,65.701477,32.488022,24.975262,43.138481,34.585754,48.008751,25.072823,46.819458
2,2016-01-06,133.018326,39.497967,50.014351,85.300003,95.625191,35.797367,91.019997,2.51,146.362579,...,19.017845,114.083321,62.297787,32.830379,24.659422,42.349586,34.154266,47.668831,24.342087,46.829220
3,2016-01-07,129.777802,38.551361,49.866177,81.919998,92.817101,35.291401,89.110001,2.28,147.983307,...,17.116064,109.300514,56.437443,32.956512,23.979164,40.678997,33.272129,46.042538,23.611343,45.405415
4,2016-01-08,129.335922,37.743946,48.506435,84.580002,91.918503,34.746506,87.849998,2.14,144.731918,...,16.866919,110.888451,54.140198,32.596134,23.298904,39.843700,32.955711,45.416023,23.345621,44.742283


As we want to compare a stock from the S&P500 index by the index itself, we need the index data as a whole by itself. We get this using the same method as earlier. We appened this to the Data frame from the compile function. 

In [199]:
# a. Printing highest observed values
def plot_w(dataframe,ticker,benchmark):
#print(data_df.loc[:, I]) # 850 rows 
    max1 = data_df.loc[:, ticker].max()
    max2 = data_df.loc[:, benchmark].max()
    print("The highest stock price observed at: \n", ticker, ":", max1.round(2), " USD on the date ", "2", 
    "\n", benchmark, ":", max2.round(2), " USD on the date", "test")

# b. Setting up plot based on dropdown input
    I = data_df.columns == ticker
    plt = dataframe.loc[:, ['Date',ticker,benchmark]].plot(x='Date', y=[ticker,benchmark], style=['-b','-k'], figsize=(8, 5), fontsize=11, legend='true')

# c. Creating the widget for the plot
widgets.interact(plot_w, 
                 dataframe = widgets.fixed(data_df), 
                 ticker = widgets.Dropdown(
                          options=data_df.columns, value='ATVI', 
                            description='Company 1:', disabled=False,),

    benchmark = widgets.Dropdown(options=data_df.columns, value='AAPL', 
                                 description='Company 2:', disabled=False,)
    )

interactive(children=(Dropdown(description='Company 1:', index=6, options=('Date', 'MMM', 'ABT', 'ABBV', 'ABMD…

<function __main__.plot_w(dataframe, ticker, benchmark)>

In [200]:
#Get sp500 index data
data_df_indexed = data_df.set_index("Date", inplace=True)
data_df_indexed = data_df/data_df.iloc[0]*100
print(data_df_indexed)

                   MMM         ABT        ABBV        ABMD         ACN  \
Date                                                                     
2016-01-04  100.000000  100.000000  100.000000  100.000000  100.000000   
2016-01-05  100.435908   99.976686   99.583409   99.718445  100.520471   
2016-01-06   98.413006   99.138113   99.600753  100.070396  100.324070   
2016-01-07   96.015519   96.762175   99.305672   96.105115   97.377994   
2016-01-08   95.688596   94.735600   96.597824   99.225720   96.435241   
2016-01-11   95.668163   94.875352   93.525439   99.718445   97.446735   
2016-01-12   95.940615   96.575803   95.191802  103.613329   99.194744   
2016-01-13   94.483023   94.419261   89.771829  100.668700   97.476191   
2016-01-14   96.158529   96.341405   95.700480  103.097137  100.049106   
2016-01-15   94.462601   95.028730  100.576715  102.545760   97.486004   
2016-01-19   94.258256   94.770864   96.454716  101.184892   99.558083   
2016-01-20   93.284279   93.669149  10

In [201]:
data_df_indexed = data_df_indexed.reset_index()





In [202]:
# a. Printing highest observed values
def plot_index(dataframe_index,ticker,benchmark):
#print(data_df.loc[:, I]) # 850 rows 
    max1 = data_df_indexed.loc[:, ticker].max()
    max2 = data_df_indexed.loc[:, benchmark].max()
    print("The highest stock price observed at: \n", ticker, ":", max1.round(2), " USD on the date ", "2", 
    "\n", benchmark, ":", max2.round(2), " USD on the date", "test")

# b. Setting up plot based on dropdown input
    I = data_df_indexed.columns == ticker
    plt = dataframe_index.loc[:, ['Date',ticker,benchmark]].plot(x='Date', y=[ticker,benchmark], style=['-b','-k'], figsize=(8, 5), fontsize=11, legend='true')

# c. Creating the widget for the plot
widgets.interact(plot_index, 
                 dataframe_index = widgets.fixed(data_df_indexed), 
                 ticker = widgets.Dropdown(
                          options=data_df.columns, value='ATVI', 
                          description='Company 1:', disabled=False,),

                benchmark = widgets.Dropdown(options=data_df_indexed.columns, value='AAPL', 
                                 description='Company 2:', disabled=False,)
    )

interactive(children=(Dropdown(description='Company 1:', index=5, options=('MMM', 'ABT', 'ABBV', 'ABMD', 'ACN'…

<function __main__.plot_index(dataframe_index, ticker, benchmark)>

In [204]:
#pct. change on the closing prices. 
data_df_pct_change = data_df.apply(lambda x: (x - x.shift(1))/x.shift(1)*100)
data_df_pct_change = data_df_pct_change.fillna(value=0)
data_df_pct_change.head() 

data_df_pct_change = data_df_pct_change.reset_index()


In [205]:
# a. Printing highest observed values
def plot_pct_change(dataframe_pct_change,ticker,benchmark):
#print(data_df.loc[:, I]) # 850 rows 
    max1 = data_df_pct_change.loc[:, ticker].max()
    max2 = data_df_pct_change.loc[:, benchmark].max()
    print("The highest percentage change: \n", ticker, ":", max1.round(2), " pct. on the date ", "2", 
    "\n", benchmark, ":", max2.round(2), " pct. on the date", "test")
# b. Setting up plot based on dropdown input
    I = data_df_pct_change.columns == ticker
    plt = dataframe_pct_change.loc[:, ['Date',ticker,benchmark]].plot(x='Date', y=[ticker,benchmark], style=['-m','-g'], figsize=(8, 5), fontsize=11, legend='true')

# c. Creating the widget for the plot
widgets.interact(plot_pct_change, 
                 dataframe_pct_change = widgets.fixed(data_df_pct_change), 
                 ticker = widgets.Dropdown(
                          options=data_df.columns, value='ATVI', 
                          description='Company 1:', disabled=False,),

                benchmark = widgets.Dropdown(options=data_df_pct_change.columns, value='AAPL', 
                                 description='Company 2:', disabled=False)
    )

interactive(children=(Dropdown(description='Company 1:', index=5, options=('MMM', 'ABT', 'ABBV', 'ABMD', 'ACN'…

<function __main__.plot_pct_change(dataframe_pct_change, ticker, benchmark)>

In [241]:
data_df_corr = data_df.pct_change().corr()
df = data_df_corr
data = df

data

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WMB,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZION,ZTS
MMM,1.000000,0.434111,0.336578,0.264108,0.498707,0.280339,0.417762,0.206384,0.166423,0.277283,...,0.223970,0.405373,0.251276,0.132721,0.362730,0.488232,0.596775,0.405610,0.340739,0.443617
ABT,0.434111,1.000000,0.457360,0.450732,0.484020,0.342146,0.493453,0.290314,0.214780,0.262426,...,0.202408,0.374291,0.226561,0.102206,0.325227,0.371582,0.439535,0.393401,0.343151,0.530334
ABBV,0.336578,0.457360,1.000000,0.306029,0.334571,0.214254,0.328809,0.165315,0.151816,0.144665,...,0.208775,0.286081,0.172011,0.059425,0.215166,0.229910,0.307586,0.219717,0.269321,0.440411
ABMD,0.264108,0.450732,0.306029,1.000000,0.407676,0.334404,0.486673,0.269574,0.148915,0.160628,...,0.256828,0.276830,0.278435,-0.014634,0.223936,0.281301,0.338265,0.325665,0.236613,0.412202
ACN,0.498707,0.484020,0.334571,0.407676,1.000000,0.393214,0.550518,0.254905,0.218867,0.298163,...,0.259396,0.454752,0.267392,0.128098,0.361623,0.414756,0.471732,0.456129,0.385275,0.485583
ATVI,0.280339,0.342146,0.214254,0.334404,0.393214,1.000000,0.511014,0.247486,0.141228,0.192126,...,0.196827,0.207240,0.193567,0.042789,0.217248,0.326734,0.237521,0.271217,0.186062,0.315614
ADBE,0.417762,0.493453,0.328809,0.486673,0.550518,0.511014,1.000000,0.317893,0.152952,0.269093,...,0.240869,0.345859,0.314146,0.108616,0.297110,0.431551,0.424962,0.470891,0.291456,0.470701
AMD,0.206384,0.290314,0.165315,0.269574,0.254905,0.247486,0.317893,1.000000,0.074364,0.153767,...,0.155693,0.205716,0.226792,0.022575,0.269514,0.350320,0.232653,0.218047,0.219562,0.237573
AAP,0.166423,0.214780,0.151816,0.148915,0.218867,0.141228,0.152952,0.074364,1.000000,0.134311,...,0.100710,0.156930,0.070762,0.065632,0.204470,0.145264,0.194174,0.195828,0.266412,0.199104
AES,0.277283,0.262426,0.144665,0.160628,0.298163,0.192126,0.269093,0.153767,0.134311,1.000000,...,0.300724,0.252278,0.131154,0.435591,0.259688,0.223627,0.235624,0.344144,0.165983,0.205129


In [304]:
def visualize_data(dataframe_corr, ticker):                
    #df.set_index('Date', inplace=True) 
    #df_corr = df.corr()
    #df_corr = df.corr()
    #I = data.columns
    #print(df_corr.head)
    
    figure = dataframe_corr.loc[:, ticker]
    #print(figure.head())
    #corr_matrix = figure.drop(figure.loc(ticker), axis=0)
    mostpos_corr = dataframe_corr.loc[:, ticker].nlargest(n=5)
    no_corr = dataframe_corr.loc[:, ticker].abs().min().round(6)
    mostneg_corr = dataframe_corr.loc[:, ticker].nsmallest(n=5)
    
    print(mostpos_corr[1:])
    
    print("", "\n", no_corr, "\n")

    print("The 5 most negatively correlated companies with", ticker, "are: \n", mostneg_corr[:])

        
# c. Creating the widget for the plot
widgets.interact(visualize_data, 
                 dataframe_corr = widgets.fixed(data), 
                 ticker = widgets.Dropdown(
                          options=data_df_corr.columns, value='MMM', 
                          description='Company 1:', disabled=False,)
                 

                )

interactive(children=(Dropdown(description='Company 1:', options=('MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI',…

<function __main__.visualize_data(dataframe_corr, ticker)>

We then want to graph a single stock and compare it to the growth of the index as a whole. We start by making a copy of the data we have, using only *"ATVI"* and the *"S&P500"* data. 

On the graph above, we see the growth in the Activision Blizzard stock compared to the S&P500 index. We see that the Activision Blizzard stock has experience a wild growth since its start in 2000. The spike around 2008 is due to the merge between Activision Entertainment and Blizzard Entertainment. The rise since ca. 2013 can be due to the rise in popularity and availability to video games. The eventual drastic fall in the end of the series comes after Blizzcon 2018 where a very unpopular annoucement was made, resulting in t